# ***Prediction models design***


## *Данный, второй документ в цикле статей описывает процесс выбора и обучения ML-модели для прогнозирования исходов теннисных матчей*


## Skills changelog:
*это первая попытка использования Machine Learning как инструмента в принципе, и поэтому
- есть только общее понимание того, что из себя представляет Data Science в целом и Machine Learning в частности;
- есть огромное желание попробовать ML на практике;
- используются любые руководства для начинающих;
- отсутствуют специфические знания и навыки, а именно - для подготовки данных, обучения моделей и т.п.;
- все, чем приходится пока руководствоваться - логика и здравый смысл.


## Sources:
- Python 3 and Pandas docs;
- StackOverflow


In [1]:
### !pip install pandas
### !pip install matplotlib
import pandas as pd
import numpy as np
#import urllib
import urllib.request
import os
import glob
import csv
import re
from datetime import datetime
from IPython.display import display
from pandas.plotting import scatter_matrix
import matplotlib
import keras

# Pandas display options tweaking
pd.options.display.max_columns = None
INIT_MAX_ROWS = 50
pd.options.display.max_rows = INIT_MAX_ROWS

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), '../../'))
print('Root directory:', ROOT_DIR)
# Downloaded files path
DOWNL_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_downloaded')
# Preprocessed files path
PREP_F_P = os.path.join(ROOT_DIR, 'match_data', 'match_data_preprocessed')

Using TensorFlow backend.


Root directory: /home/vl/Repos/machine-learning-on-tennis


In [2]:
preprocessed_data_file = 'atp_matches_1991-2018_preprocessed_final.csv'
preprocessed_data_full_path = os.path.join(PREP_F_P, preprocessed_data_file)
preprocessed_data = pd.read_csv(preprocessed_data_full_path, header=0, encoding='utf-8-sig', engine='python')

display(preprocessed_data)

,ht,age,rank,rank_points,ace_last,ace_max,ace_mean,ace_min,df_last,df_max,df_mean,df_min,perc_1stIn_last,perc_1stIn_max,perc_1stIn_mean,perc_1stIn_min,perc_w_1stIn_last,perc_w_1stIn_max,perc_w_1stIn_mean,perc_w_1stIn_min,perc_w_2ndIn_last,perc_w_2ndIn_max,perc_w_2ndIn_mean,perc_w_2ndIn_min,winner_flag
0,-0.1000,-0.3333,0.0769,-0.0004,0.2000,-0.3333,0.0000,1.0000,0.0000,1.0000,1.0000,0.0000,-0.0909,-0.1667,-0.1250,-0.1429,-0.3333,0.0000,-0.2500,-0.1000,-0.0769,-0.3333,-0.1000,-0.0714,-1
1,0.1000,-0.2000,-0.0345,0.0002,0.0909,0.0909,0.1250,0.1667,-0.1250,-0.1429,-0.1667,-0.2500,0.1000,0.1250,0.1111,0.0588,0.0500,0.1111,0.0714,0.0625,0.0476,0.0833,0.0667,0.0625,1
2,0.3333,-0.5000,-0.0303,0.0007,-0.0625,-0.0833,-0.1000,-0.2000,-0.2000,-0.5000,-1.0000,-0.5000,0.0714,0.0625,0.0588,0.0556,-0.1111,-0.1250,-0.0909,-0.1111,0.1429,0.5000,0.1000,0.0714,1
3,0.3333,-0.1667,0.1250,-0.0003,-1.0000,-0.2500,-0.2500,-0.3333,0.5000,0.0000,0.0000,-1.0000,-0.1250,-0.1250,-0.2500,-0.3333,-0.3333,-0.0769,-0.1000,-0.1000,-0.0370,-0.1250,-0.0476,-0.0385,1
4,-0.0455,-0.1000,0.1111,-0.0003,-1.0000,-0.1667,-0.2000,-0.2500,1.0000,0.0000,0.0000,1.0000,0.2000,0.1667,0.2000,0.2000,-0.1111,-0.2500,-0.1111,-0.0714,-0.0435,-0.0526,-0.0625,-0.0625,-1
5,0.3333,1.0000,-0.0147,0.0007,-0.5000,0.3333,-1.0000,-0.3333,-0.3333,-1.0000,-0.5000,-0.3333,0.0417,0.0455,0.0769,0.2000,0.0000,0.0000,0.1250,0.0714,-0.1667,-0.1667,-0.2000,-0.3333,-1
6,0.1250,0.1667,-0.0303,0.0022,1.0000,-0.2000,-0.3333,-0.3333,0.0000,0.0000,-1.0000,-1.0000,0.0500,0.0909,0.0833,0.0769,-0.0833,-0.1111,-0.0909,-0.0833,-0.0357,-0.1429,-0.1000,-0.0667,1
7,-0.3333,0.1250,0.0400,-0.0013,-0.0333,-0.0417,-0.0588,-0.1667,1.0000,1.0000,-0.5000,-0.3333,-0.0667,-0.1429,-0.0769,-0.0667,-0.1667,-0.5000,-0.2000,-0.1667,1.0000,-0.1000,-0.0909,-0.0556,-1
8,-0.3333,0.1667,-0.0169,0.0001,0.0000,0.2000,0.5000,1.0000,-0.3333,-0.3333,-0.3333,-0.5000,-0.2000,-0.2000,1.0000,0.0714,-0.2000,0.3333,0.2500,0.2000,-0.3333,0.1429,0.0909,0.0526,1
9,-0.1000,0.5000,0.0625,-0.0003,-0.1429,-0.0769,-0.1429,-0.2500,-1.0000,0.5000,1.0000,-1.0000,-0.2500,0.2000,0.2500,0.1250,-0.1667,-0.2500,-0.2500,-0.2000,0.0714,0.2500,-0.2000,-0.2000,-1


Разделим данные на обучающую и тестовую выборку:

In [3]:
def _feat_target_sep(data, res_col):
    features = data.drop([res_col],1)
    target = data[res_col]
    return features, target

def _standardise_data(features):
    # Standardising the data.
    from sklearn.preprocessing import scale
    #Center to the mean and component wise scale to unit
    #variance.
    cols = features.columns

    for col in cols:
        features[col] = scale(features[col])

    return features

def split_data(data, test_part):
    # Split into feature set and target variable
    # res = 1 if left player wins,
    # res = -1 - if the right one does.
    X_all, y_all = _feat_target_sep(data, 'winner_flag')

#    pd.options.display.max_rows=50
#    np.set_printoptions(threshold=np.inf)
    X_all = _standardise_data(X_all)
#    y_all = _encode_target(y_all.values)
#    y_all = _relabel(y_all)

    total_matches = X_all.shape[0]
    test_num = total_matches * test_part
    test_num = int(test_num)
    train_num = total_matches - test_num

    X_train = X_all[:train_num]
    y_train = y_all[:train_num]
#    y_train = _encode_target(y_train.values)

    X_test = X_all[train_num:]
    y_test = y_all[train_num:]
#    y_test = _relabel(y_test)

#    y_train = _encode_target(y_train.values)

    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = split_data(preprocessed_data, 0.15)

display(X_train)
display(y_train)
display(X_test)
display(y_test)

,ht,age,rank,rank_points,ace_last,ace_max,ace_mean,ace_min,df_last,df_max,df_mean,df_min,perc_1stIn_last,perc_1stIn_max,perc_1stIn_mean,perc_1stIn_min,perc_w_1stIn_last,perc_w_1stIn_max,perc_w_1stIn_mean,perc_w_1stIn_min,perc_w_2ndIn_last,perc_w_2ndIn_max,perc_w_2ndIn_mean,perc_w_2ndIn_min
0,-0.442336,-0.671580,0.414856,-0.012476,0.427370,-0.772759,-0.005932,1.733429,-0.007321,1.748359,1.484821,0.003200,-0.282142,-0.475974,-0.319482,-0.412089,-1.020474,0.009506,-0.628334,-0.287763,-0.266453,-1.068444,-0.243100,-0.219242
1,0.443135,-0.401907,-0.184089,0.003866,0.193609,0.206314,0.233563,0.290812,-0.217457,-0.251845,-0.269308,-0.372852,0.304335,0.365181,0.306436,0.157049,0.157398,0.321634,0.187690,0.193902,0.166856,0.274001,0.171418,0.193981
2,1.476036,-1.008823,-0.161507,0.017484,-0.135071,-0.195747,-0.197528,-0.344022,-0.343539,-0.876810,-1.522172,-0.748903,0.216471,0.184954,0.167785,0.148020,-0.337658,-0.341673,-0.224384,-0.320665,0.498537,1.616768,0.254222,0.221447
3,1.476036,-0.334539,0.673466,-0.009753,-2.143786,-0.580499,-0.484923,-0.574793,0.833227,-0.001754,-0.018675,-1.501007,-0.386903,-0.355727,-0.650866,-0.949342,-1.020474,-0.206539,-0.247489,-0.287763,-0.127585,-0.397222,-0.112802,-0.117711
4,-0.201045,-0.199601,0.598733,-0.009753,-2.143786,-0.388238,-0.389125,-0.430583,1.673774,-0.001754,-0.018675,1.507406,0.611552,0.485428,0.542116,0.555474,-0.337658,-0.692852,-0.275671,-0.202990,-0.150208,-0.163921,-0.149852,-0.191776
5,1.476036,2.025759,-0.077634,0.017484,-1.072471,0.765785,-1.921894,-0.574793,-0.567629,-1.751867,-0.770423,-0.498152,0.125227,0.135932,0.215769,0.555474,0.003749,0.009506,0.323778,0.220282,-0.578992,-0.531595,-0.491761,-1.027481
6,0.553818,0.339948,-0.161507,0.058339,2.141474,-0.465096,-0.644522,-0.574793,-0.007321,-0.001754,-1.522172,-1.501007,0.150726,0.266849,0.232736,0.208122,-0.252230,-0.302622,-0.224384,-0.238263,-0.123061,-0.454902,-0.243100,-0.204738
7,-1.475237,0.255586,0.216462,-0.036989,-0.072506,-0.099733,-0.118591,-0.286373,1.673774,1.748359,-0.770423,-0.498152,-0.207795,-0.407344,-0.191966,-0.197075,-0.508516,-1.395209,-0.501385,-0.485469,3.481578,-0.316662,-0.220472,-0.170482
8,-1.475237,0.339948,-0.089462,0.001142,-0.001156,0.458122,0.952048,1.733429,-0.567629,-0.585066,-0.519790,-0.748903,-0.617315,-0.571999,2.662973,0.192603,-0.610846,0.945889,0.641149,0.601465,-1.158825,0.466055,0.231594,0.163429
9,-0.442336,1.014232,0.337434,-0.009753,-0.307338,-0.180976,-0.279723,-0.430583,-1.688415,0.873303,1.484821,-1.501007,-0.770924,0.581453,0.674670,0.343846,-0.508516,-0.692852,-0.628334,-0.584173,0.249689,0.811172,-0.491761,-0.616109


0       -1
1        1
2        1
3        1
4       -1
5       -1
6        1
7       -1
8        1
9       -1
10      -1
11       1
12       1
13       1
14       1
15      -1
16       1
17      -1
18      -1
19       1
20       1
21      -1
22      -1
23       1
24       1
        ..
14247    1
14248   -1
14249    1
14250    1
14251   -1
14252    1
14253   -1
14254   -1
14255    1
14256    1
14257    1
14258    1
14259   -1
14260    1
14261   -1
14262    1
14263   -1
14264    1
14265   -1
14266   -1
14267    1
14268   -1
14269    1
14270   -1
14271    1
Name: winner_flag, Length: 14272, dtype: int64

,ht,age,rank,rank_points,ace_last,ace_max,ace_mean,ace_min,df_last,df_max,df_mean,df_min,perc_1stIn_last,perc_1stIn_max,perc_1stIn_mean,perc_1stIn_min,perc_w_1stIn_last,perc_w_1stIn_max,perc_w_1stIn_mean,perc_w_1stIn_min,perc_w_2ndIn_last,perc_w_2ndIn_max,perc_w_2ndIn_mean,perc_w_2ndIn_min
14272,-1.475237,-0.286390,0.040650,-0.031542,-1.072471,-2.311533,-0.963913,-0.863385,1.673774,-1.751867,-1.522172,0.003200,0.233368,0.124974,0.151348,0.192603,0.311047,0.477838,-2.532558,-0.219293,0.001189,-1.605616,-0.823226,-1.027481
14273,-0.553019,0.508468,-0.123872,0.148220,0.236890,2.304559,1.910029,-0.863385,-0.007321,0.435775,0.733073,1.507406,0.253030,-2.878903,2.662973,2.812841,-0.232563,0.290449,-2.532558,0.008646,0.001189,0.327815,2.492171,-0.341759
14274,-1.475237,0.407315,-0.019029,0.041997,-0.429682,2.304559,-0.005932,0.002220,0.833227,0.581559,-1.522172,-1.501007,-0.258793,2.888357,-0.366936,-0.243915,-0.508516,0.571392,-1.263075,-0.414923,0.268831,0.253378,0.231594,0.309708
14275,-0.340064,0.227466,0.060005,-0.064226,-0.165924,-0.333307,-0.389125,-0.430583,0.552988,-1.751867,1.484821,0.003200,0.253030,-0.716180,-0.650866,-0.008867,-0.157889,-0.341673,-0.416838,-0.979287,3.481578,0.408374,0.502883,3.087162
14276,0.885870,-0.165816,-0.314738,0.494125,0.141757,0.161307,0.185664,0.290812,0.552988,0.435775,0.357199,1.507406,-0.282142,-0.571999,-0.650866,-0.949342,0.345157,0.321634,0.369226,0.996579,-0.157169,-0.141042,-0.102606,-0.109070
14277,1.476036,0.291799,-0.058278,0.041997,-1.072471,-0.772759,-0.325323,-0.190118,-0.247549,-0.293498,-0.394549,0.003200,0.436131,0.244933,0.453836,0.696560,-0.435379,-0.146698,-0.134759,-0.139559,1.161203,0.811172,0.627214,0.618314
14278,-0.553019,0.676989,0.091189,-0.107804,-0.536814,-1.157510,-1.921894,0.002220,-0.567629,-0.001754,-1.522172,-0.748903,-0.222235,-0.283636,-0.208932,-0.210337,0.516014,0.945889,0.514201,0.502761,0.184258,0.206975,0.171418,0.182563
14279,-0.885071,2.025759,0.180439,-0.009753,-0.195921,-0.124890,-0.180093,-0.344022,-0.007321,-0.293498,-0.770423,-1.501007,0.233368,0.087199,0.232736,0.247626,0.618345,-0.224520,-0.205088,-0.147265,-3.479200,-0.352431,-0.305265,-0.064631
14280,-0.885071,1.014232,-0.357213,0.120983,-0.358333,-0.333307,-0.325323,-0.430583,-0.007321,-0.439282,-0.770423,0.003200,-1.538966,0.104212,0.895504,1.401987,0.283083,-1.395209,-0.839830,-0.238263,0.166856,0.363583,-1.237744,-0.084690
14281,0.000399,2.025759,1.345532,-0.012476,-0.429682,-0.157434,-0.279723,-0.863385,-1.688415,-0.585066,-0.018675,0.003200,0.158714,0.965841,1.337438,0.696560,-1.020474,-0.224520,-0.224384,-0.219293,0.871286,-0.454902,2.492171,0.515549


14272   -1
14273   -1
14274    1
14275   -1
14276   -1
14277   -1
14278    1
14279   -1
14280    1
14281   -1
14282    1
14283   -1
14284    1
14285   -1
14286    1
14287   -1
14288    1
14289   -1
14290    1
14291   -1
14292    1
14293   -1
14294   -1
14295   -1
14296    1
        ..
16765   -1
16766    1
16767   -1
16768    1
16769   -1
16770    1
16771   -1
16772    1
16773   -1
16774    1
16775   -1
16776    1
16777   -1
16778    1
16779    1
16780   -1
16781    1
16782    1
16783    1
16784    1
16785   -1
16786   -1
16787   -1
16788    1
16789   -1
Name: winner_flag, Length: 2518, dtype: int64

Обучение нейронной сети:

In [4]:
def get_model(
        optimizer,
        loss):

    from keras.models import Sequential
    import keras.layers as ll
    from keras import regularizers

    model = Sequential(name="mlp")
    #model = Sequential()

    # input layer
    inputs = ll.Dense(
                 24,
                 kernel_regularizer=regularizers.l2(0.001),
                 activation='relu',
                 input_shape=(24, ) )
    #flatten = ll.Flatten()

    # network body
    layer1 = ll.Dense(24, kernel_regularizer=regularizers.l2(0.001))
    activ = ll.Activation('relu')
    dropout = ll.Dropout(0.15)
    layer2 = ll.Dense(24, kernel_regularizer=regularizers.l2(0.001))
    layer3 = ll.Dense(24, kernel_regularizer=regularizers.l2(0.001))
    layer4 = ll.Dense(24, kernel_regularizer=regularizers.l2(0.001))

    # output layer
    output = ll.Dense(2, kernel_regularizer=regularizers.l2(0.001), activation='sigmoid')

    # model constructing
    model.add(inputs)

    model.add(layer1)
    model.add(activ)
    model.add(dropout)

    model.add(layer2)
    model.add(activ)
    model.add(dropout)

    model.add(layer3)
    model.add(activ)
    model.add(dropout)

    model.add(layer4)
    model.add(activ)
    model.add(dropout)

    model.add(output)

    # categorical_crossentropy is our good old
    # crossentropy but applied for one-hot-encoded
    # vectors
    from keras.metrics import categorical_accuracy
    model.compile(
       optimizer=optimizer,
       loss=loss,
#       lr = 0.0001,
       metrics=[categorical_accuracy])
#       metrics='accuracy')

    return model

from sklearn.metrics import f1_score
def train_classifier(
        model,
        train_features,
        train_target,
        batch_size=512,
        epochs=100):

    model.fit(
        train_features,
        train_target,
        batch_size,
        epochs)
    return model

def test_classifier_f1(
        model,
        test_features,
        test_target):

    from sklearn.metrics import f1_score
    print('Testing model...')
    target_pred = model.predict_classes(test_features)
    _relabel_output(target_pred)
    test_target = test_target.values

    print('Predicted target:')
    print(target_pred)
    print('Test target:')
    print(test_target)

    f1 = f1_score(
             test_target,
             target_pred,
             average='macro')
    print( 'sum =', sum(test_target == target_pred) )
    print( 'target size =', len(target_pred) )
    acc = sum(test_target == target_pred) / float(len(target_pred))
    return f1, acc

def train_nn(data):
    X_train, y_train, X_test, y_test = split_data(data, 0.15)
    
    #learning_rate = 0.0001
    opt = keras.optimizers.Adam(lr=0.001)
    #opt = keras.optimizers.RMSprop(lr=0.008)
    #opt = keras.optimizers.Adagrad(lr=0.008)
    #opt = keras.optimizers.Adadelta(lr=1.0)
    #loss = 'binary_crossentropy'
    loss = 'categorical_crossentropy'
    
    model = get_model(
                opt,
                loss)
    k = 5
    batch_size = 256
    epochs = 110
    #model = train_k_fold(model, X_train, y_train, k, batch_size, epochs)
    
    model = get_model(
                opt,
                loss)
    model = train_classifier(model, X_train, y_train, batch_size, epochs)
    
    f1, acc = test_classifier_f1(model, X_test, y_test)
    print (f"F1 score and accuracy score for test set: {f1:.4f} , {acc:.4f}.")
    
train_nn(preprocessed_data)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


ValueError: Error when checking target: expected dense_12 to have shape (2,) but got array with shape (1,)